In [1]:
import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame

In [2]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.5


In [3]:
val inputStreamPath = "/home/jovyan/data/events-stream"
val modelPath = "/home/jovyan/models/spark-ml-model"

val dataSchema = new StructType()
    .add("tweet", StringType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)

inputStreamPath = /home/jovyan/data/events-stream
modelPath = /home/jovyan/models/spark-ml-model
dataSchema = StructType(StructField(tweet,StringType,true))
inputDF = [tweet: string]


[tweet: string]

In [4]:
val getProbability = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(1))

val predictionsDF: DataFrame = inputDF

val clf = PipelineModel.load("/home/jovyan/models/spark-ml-model")

predictionsDF.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>
    clf
    .transform(batchDF)
    .select(getProbability('probability).alias("probability_positive"), 'prediction, 'tweet)
    .show
}.start()

getProbability = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))
predictionsDF = [tweet: string]
clf = pipeline_72e0eba779e6


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@6a2b9b1f

+--------------------+----------+--------------------+
|probability_positive|prediction|               tweet|
+--------------------+----------+--------------------+
|  0.5144744106785156|       1.0|@nick_carter THE ...|
| 0.49870081217915585|       0.0|OMG im getting si...|
|  0.4998981906985482|       0.0|A great coffee th...|
|  0.5172676433669997|       1.0|@TheMandyMoore aw...|
|  0.4956454180720865|       0.0|Scottish radio un...|
|  0.5301574183838913|       1.0|@fijjybutt get ea...|
|  0.5215151259185439|       1.0|Tackling the insp...|
| 0.46344600013957865|       0.0|@dbrad86 very sor...|
|  0.5144744106785156|       1.0|@OhFerras @MDMOLI...|
|  0.5183223687889681|       1.0|So tired now. Bum...|
|  0.5142174137459792|       1.0|@Journi Sorry - l...|
|  0.4823860400233243|       0.0|I'm craving Sushi...|
|  0.5618328516937856|       1.0|Dear Coachella, I...|
|   0.536445520749846|       1.0|@BlueEyed_Cat the...|
|  0.5616812248098265|       1.0|@gerandclaire Aft...|
| 0.434957

In [5]:
spark.stop()